In [220]:
import numpy as np
import qutip as qt
import pickle
from qiskit.quantum_info import DensityMatrix, Statevector, partial_trace
from scipy.linalg import logm, expm


import sys
sys.path.append('/Users/bence/code/liouvillian_metro/')

from tools.classical import HamHam, find_ideal_heisenberg

In [221]:
#TODO: Check with bound on sum_w A A^\dagger for unitary A, if the oft is calculated correctly.


In [222]:
def oft(jump_op: np.ndarray, energy: float, num_labels: int, 
                          hamiltonian: np.ndarray, sigma: float) -> np.ndarray:
    
    time_labels = np.arange(-0.5, 0.5, 1/num_labels)
    
    gauss = lambda t: np.exp(-(t ** 2) / (4 * sigma ** 2))
    gauss_normalization = np.sum([np.sqrt(np.abs(gauss(t))**2) for t in time_labels])
    time_evolution = lambda t: expm(1j * t * hamiltonian)
    
    oft_op = np.zeros_like(hamiltonian, dtype=np.complex128)
    for t in time_labels:
        oft_op += np.exp(1j * 2 * np.pi * energy * t / num_labels) * gauss(t) * time_evolution(t) @ jump_op @ time_evolution(-t)
        
    return oft_op / (np.sqrt(num_labels) * gauss_normalization)
    

In [223]:
num_energy_bits = 6
num_qubits = 3
bohr_bound = 2 ** (-num_energy_bits + 1)
eps = 0.05
sigma = 10

jump_op = qt.sigmax()
jump_op = qt.tensor([qt.qeye(2), jump_op, qt.qeye(2)]).full()
hamiltonian = find_ideal_heisenberg(num_qubits, bohr_bound, eps, signed=False, for_oft=True)


oft_op = oft(jump_op, 0.4, 2**num_energy_bits, hamiltonian.qt.full(), sigma)

Original spectrum:  [-3.5    -2.9921 -2.3725 -1.5     1.6025  2.1     2.8895  3.7725]
Ideal spectrum:  [-0.0071  0.0255  0.0653  0.1214  0.3209  0.3529  0.4036  0.4604]
Nonrescaled coefficients:  [0.9 0.7 0.9 1. ]
Rescaled coefficients:  [0.05785714 0.045      0.05785714 0.06428571]


In [224]:
time_labels = np.arange(-0.5, 0.5, 1/2**num_energy_bits)
energy_labels = 2*np.pi / (time_labels * 2**num_energy_bits)
energy_labels[int(2**num_energy_bits / 2)] = 0

big_sum = np.zeros_like(hamiltonian.qt.full(), dtype=np.complex128)
for w in energy_labels:
    oft_op = oft(jump_op, w, 2**num_energy_bits, hamiltonian.qt.full(), sigma)
    big_sum += oft_op @ oft_op.conj().T
    

/var/folders/pg/6vn02n251m14stcv7vg03pjr0000gn/T/ipykernel_20927/2763597389.py:2: RuntimeWarning: divide by zero encountered in divide
  energy_labels = 2*np.pi / (time_labels * 2**num_energy_bits)


In [225]:
gauss = lambda t: np.exp(-(t ** 2) / (4 * sigma ** 2))
gauss_normalization = np.sum([np.sqrt(gauss(t)**2) for t in time_labels])

gauss_squared_sum = np.sum([gauss(t)**2 for t in time_labels]) / gauss_normalization
gauss_squared_sum

0.9997916388571882

In [229]:
big_sum_norm = np.linalg.norm(big_sum)
print(big_sum_norm)
# Test with random matrix
random_matrix = np.random.rand(2**num_qubits, 2**num_qubits) + 1j * np.random.rand(2**num_qubits, 2**num_qubits)
random_matrix /= np.linalg.norm(random_matrix)
random_matrix *= big_sum_norm

dist = np.linalg.norm(big_sum - np.eye(2**num_qubits)*gauss_squared_sum)
print(f'Absolute distance to wanted result: {dist}')
print(f'Relative distance: {dist / np.linalg.norm(big_sum)}')
dist2 = np.linalg.norm(random_matrix - np.eye(2**num_qubits)*gauss_squared_sum)
print(f'Distance with random matrix: {dist2}')

2.8100746983237292
Absolute distance to wanted result: 0.02266644642332293
Relative distance: 0.008066136617949679
Distance with random matrix: 3.4479062230035353
